In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_pickle('100_fingerprints_expanded_dataset.pkl')

In [3]:
# First stage
# Divide into training and test sets
X = list(df["Image"])
y = df["Hand"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
model1 = RandomForestClassifier()
model1.fit(X_train,y_train) # Fit the model fo training data

RandomForestClassifier()

In [4]:
dflh = df[df.Hand==0]
dfrh = df[df.Hand==1]

In [5]:
# Left hand training
Xlh = list(dflh["Image"])
ylh = dflh["Identity"]
X_trainlh, X_testlh, y_trainlh, y_testlh = train_test_split(Xlh,ylh,test_size=0.2)
lh_model = RandomForestClassifier()
lh_model.fit(X_trainlh,y_trainlh)

# Right hand training
Xrh = list(dfrh["Image"])
yrh = dfrh["Identity"]
X_trainrh, X_testrh, y_trainrh, y_testrh = train_test_split(Xrh,yrh,test_size=0.2)
rh_model = RandomForestClassifier()
rh_model.fit(X_trainrh,y_trainrh)

RandomForestClassifier()

In [17]:
pred = model1.predict(X_test)

In [19]:
print(pred)
print(np.array(y_test))

[0 1 0 ... 0 0 0]
[0 0 0 ... 0 0 1]


In [31]:
X_lh = []
y_lh = []
X_rh = []
y_rh = []

for i in range(len(pred)):
    if pred[i]==0:
        X_lh.append(X_test[i])
        j = y_test.index[i]
        y_lh.append(df["Identity"][j])
    else:
        X_rh.append(X_test[i])
        j = y_test.index[i]
        y_rh.append(df["Identity"][j])

In [34]:
pred_lh = lh_model.predict(X_lh)
pred_rh = rh_model.predict(X_rh)

In [36]:
# Calculate accuracy
len_lh = len(pred_lh)
len_rh = len(pred_rh)
score_lh = 0
score_rh = 0
for i in range(len_lh):
    if pred_lh[i] == y_lh[i]:
        score_lh += 1/len_lh
for i in range(len_rh):
    if pred_rh[i] == y_rh[i]:
        score_rh += 1/len_rh

In [37]:
print(score_lh)
print(score_rh)

0.6455566905005107
0.6238981390793391
